Alexnet的网络结构

<img src="https://ws4.sinaimg.cn/large/006tKfTcgy1fodv2ppd85j307l0n13yk.jpg">

可以看出`AlexNet`就是几个卷积池化堆叠后连接几个全连接层, 下面就让我们来尝试仿照这个结构来解决[cifar10](https://www.cs.toronto.edu/~kriz/cifar.html)分类问题

In [10]:
import torch as t
import numpy as np
from torch import nn
from torchvision.datasets import CIFAR10

In [17]:
class Alexnet(nn.Module):
    def __init__(self):
        super(Alexnet, self).__init__()
        
        # nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0,
        # dilation=1, groups=1, bias=True, padding_mode='zeros',)
        self.conv1 = nn.Sequential(nn.Conv2d(3, 64, 5), nn.ReLU(True))
        self.maxpool1 = nn.MaxPool2d(3, 2)
        self.conv2 = nn.Sequential(nn.Conv2d(64, 64, 5, 1), nn.ReLU(True))
        self.maxpool2 = nn.MaxPool2d(3, 2)
        self.fc1 = nn.Sequential(nn.Linear(1024, 384), nn.ReLU(True))
        self.fc2 = nn.Sequential(nn.Linear(384, 192), nn.ReLU(True))
        self.fc3 = nn.Linear(192, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        # 拉平矩阵
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x

In [18]:
alexnet = Alexnet()
alexnet

Alexnet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Sequential(
    (0): Linear(in_features=1024, out_features=384, bias=True)
    (1): ReLU(inplace)
  )
  (fc2): Sequential(
    (0): Linear(in_features=384, out_features=192, bias=True)
    (1): ReLU(inplace)
  )
  (fc3): Linear(in_features=192, out_features=10, bias=True)
)

In [19]:
test_input = t.zeros((1, 3, 32, 32), requires_grad=True)
out = alexnet(test_input)
out.shape

torch.Size([1, 10])

In [ ]:
# training
from torch.utils.data import DataLoader

def transform():
    x = np.array(x, type='float32') / 255
    x = (x - 0.5) / 0.5  # 标准化
    x = x.transpose((2, 0, 1))  # channel first
    x = t.from_numpy(x)
    return x

# 下载CIFAR10 datasets
trainset = CIFAR10('./data', train=True, transform=transform, download=True)
train_data = DataLoader(trainset, batch_size=64, shuffle=True)
testset = CIFAR10('./data', train=True, transform=transform, shuffle=True)
test_data = DataLoader(testset, batch_size=64, shuffle=True)

net = Alexnet().cuda()
optimizer = t.optim.SGD(net.parameters, lr=1e-1)
criterion = nn.CrossEntropyLoss()

In [ ]:
from utils import train

train(net, train_data, test_data, 20, optimizer, criterion)

### unsolved-error:   
IndexError: invalid index of a 0-dim tensor. Use tensor.item() to convert a 0-dim tensor to a Python    

``` 
train_loss += loss.data[0]
```
将其改为：  `train_loss += loss.data[0]` 后依然报错
